In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO

model=YOLO('yolo11n.pt')

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker

model = YOLO('yolo11n.pt')

class_list = model.names  
#class_list = ['person', 'bicycle', 'car', 'motorcycl
# e', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

cap = cv2.VideoCapture('indian-traffic.mp4')

tracker = Tracker()

counted_ids_red_to_blue = set()  # Set to keep track of counted IDs for red to blue
counted_ids_blue_to_red = set()  # Set to keep track of counted IDs for blue to red

count_red_to_blue = 0  # Counter for objects crossing from red to blue
count_blue_to_red = 0  # Counter for objects crossing from blue to red

line_y_red = 230  # Red line position
line_y_blue = line_y_red + 150  # Blue line position

while True:
    ret, frame = cap.read()
    if not ret:
        break

    #frame = cv2.resize(frame, (1020, 500))
    results = model.predict(frame)
    
    if results[0].boxes.data is not None:
        a = results[0].boxes.data.detach().cpu().numpy()
        px = pd.DataFrame(a)

        list = []
        for index, row in px.iterrows():
            x1, y1, x2, y2, conf, d = map(int, row)
            c = class_list[d]
            if 'car' in c:
                list.append([x1, y1, x2, y2])

        bbox_id = tracker.update(list)

        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            
            # Draw the lines on the frame
            cv2.line(frame, (5, line_y_red), (1280, line_y_red), (0, 0, 255), 1) 
            cv2.putText(frame, 'Red Line', (5, line_y_red - 10), cv2.FONT_HERSHEY_DUPLEX , 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.line(frame, (5, line_y_blue), (1280, line_y_blue), (255, 0, 0), 1) 
            cv2.putText(frame, 'Blue Line', (5, line_y_blue - 10), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

            # Draw dot in the center and write tracking id on it
            cx = (int(x3) + int(x4)) // 2
            cy = (int(y3) + int(y4)) // 2
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # Red dot
            cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # Draw the bounding box 
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Green rectangle
            
            # Check if the object has crossed the red line to blue line
            if line_y_red - 5 <= cy <= line_y_red + 5:
                if id not in counted_ids_red_to_blue:
                    counted_ids_red_to_blue.add(id)
                    count_red_to_blue += 1

            # Check if the object has crossed the blue line to red line
            if line_y_blue - 5 <= cy <= line_y_blue + 5:
                if id not in counted_ids_blue_to_red:
                    counted_ids_blue_to_red.add(id)
                    count_blue_to_red += 1

    # Display the total counts on the frame
    cv2.putText(frame, f'Red to Blue Count (Down): {count_red_to_blue}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Blue to Red Count (Up): {count_blue_to_red}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 12 cars, 3 motorcycles, 1 bus, 8 trucks, 123.4ms
Speed: 12.6ms preprocess, 123.4ms inference, 8.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 12 cars, 3 motorcycles, 1 bus, 8 trucks, 71.9ms
Speed: 5.0ms preprocess, 71.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 13 cars, 2 motorcycles, 2 buss, 6 trucks, 108.3ms
Speed: 3.0ms preprocess, 108.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 13 cars, 2 motorcycles, 1 bus, 7 trucks, 85.6ms
Speed: 4.0ms preprocess, 85.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 13 cars, 2 motorcycles, 7 trucks, 95.0ms
Speed: 3.0ms preprocess, 95.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 13 cars, 3 motorcycles, 1 bus, 7 trucks, 68.4ms
Speed: 3.0ms preprocess, 68.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384